## Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is essentially the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD can come from any data source, e.g. text files, a database, a JSON file, etc.


## Objectives

You will be able to:

- Apply the map(func) transformation to a given function on all elements of an RDD in different partitions 
- Apply a map transformation for all elements of an RDD 
- Compare the difference between a transformation and an action within RDDs 
- Use collect(), count(), and take() actions to trigger spark transformations  
- Use filter to select data that meets certain specifications within an RDD 
- Set number of partitions for parallelizing RDDs 
- Create RDDs from Python collections 


## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

Resilient: RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies while performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

Distributed: The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

Dataset: The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high-level Spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 



Key Characteristics of RDDs:

- Immutable: Once an RDD is created, it cannot be modified. 
- Lazily Evaluated: RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- In-Memory: The operations in Spark are performed in-memory rather than in the database. This is what allows Spark to perform fast operations with very large quantities of data.




### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="./images/rdd_diagram.png" width=500>

### Transformations

Transformations create a new dataset from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base dataset (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation is an RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check: confirm the length of the list (it should be 1000)

EVERYTHING DONE ON GOOGLE COLAB

In [1]:
# innstall java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://downloads.apache.org/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz

# unzip the spark file to the current folder
!tar xf spark-2.4.7-bin-hadoop2.7.tgz

# install findspark using pip
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-1.8.0-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

import findspark
findspark.init()
import pyspark

In [2]:
data = list(range(1,1001))
len(data)

# 1000

1000

### Initialize an RDD

When using Spark to make computations, datasets are treated as lists of entries. Those lists are split into different partitions across different cores or different computers. Each list of data held in memory is a partition of the RDD. The reason why Spark is able to make computations far faster than other big data processing languages is that it allows all data to be stored __in-memory__, which allows for easy access to the data and, in turn, high-speed processing. Here is an example of how the alphabet might be split into different RDDs and held across a distributed collection of nodes:

<img src ="./images/partitions_1.png" width ="500">  
To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use `'local[*]'` as the master.

In [3]:
sc = pyspark.SparkContext('local[*]')

Once you've created the SparkContext, you can use the `.parallelize()` method to create an RDD that will distribute the list of numbers across multiple cores. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [4]:
rdd = sc.parallelize(c=data, numSlices=10)
print(type(rdd))
# <class 'pyspark.rdd.RDD'>

<class 'pyspark.rdd.RDD'>


Determine how many partitions are being used with this RDD with the `.getNumPartitions()` method.

In [5]:
rdd.getNumPartitions()
# 10

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first `n` items in the RDD
* `top`: returns the top `n` items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [6]:
# count
rdd.count()

1000

In [7]:
# first
rdd.first()

1

In [10]:
# take
rdd.take(3)

[1, 2, 3]

In [11]:
# top
rdd.top(3)

[1000, 999, 998]

In [12]:
# collect
rdd.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

## Map functions

Now that you've been working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new e-commerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 with a random number from 0-1.

In [13]:
import random
import numpy as np

nums = np.array(range(1, 1001))
sales_figures = nums * np.random.rand(1000)
sales_figures

array([3.43442108e-01, 1.39464323e+00, 2.19310923e+00, 5.92328190e-01,
       4.26571479e+00, 5.02275073e+00, 2.75055404e+00, 1.95750060e+00,
       6.92977035e+00, 9.02904975e+00, 5.34404486e+00, 6.99274681e+00,
       6.75839725e+00, 7.73004694e+00, 4.82424827e+00, 1.38904217e+01,
       7.22181209e+00, 4.05045269e+00, 5.96145067e+00, 1.71844328e+01,
       1.05201555e+00, 1.95779126e+01, 4.51802271e+00, 2.37604039e+00,
       8.10690775e+00, 8.27292531e+00, 4.55045966e+00, 4.14572875e+00,
       1.94602577e+01, 2.37904243e+01, 1.98736256e+01, 1.55840456e+01,
       4.12976816e+00, 2.60410865e+01, 4.76585842e+00, 3.15221933e+01,
       2.13193316e+00, 3.37601250e+01, 2.79456864e+00, 6.19725720e+00,
       3.90813467e+01, 1.33182992e+01, 4.17628185e+01, 1.87872695e+01,
       2.57503450e+01, 2.35329070e+01, 8.28614780e+00, 4.05535014e+00,
       1.79377821e+00, 4.03310628e+01, 2.89823862e+01, 2.42491359e+01,
       2.76312961e+01, 1.85753799e+01, 4.08723815e+01, 5.11236693e+01,
      

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [15]:
price_items = sc.parallelize(c=sales_figures, numSlices=10)
price_items.take(4)

[0.34344210786439855,
 1.3946432333499241,
 2.1931092282021343,
 0.5923281903233097]

Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied (assume a sales tax of 8%). To make this happen, create a function called `sales_tax()` that returns the amount of money our company will receive after the sales tax has been applied. The function will have this parameter:

* `item`: (float) number to be multiplied by the sales tax.


Apply that function to the rdd by using the `.map()` method and assign it to a variable `renenue_minus_tax`

In [18]:
def sales_tax(num):
    return num*0.92

revenue_minus_tax = price_items.map(sales_tax)

Remember, Spark has __lazy evaluation__, which means that the `sales_tax()` function is a transformer that is not executed until you call an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` rdd.

In [19]:
# perform action to retrieve rdd values
revenue_minus_tax.take(10)

[0.31596673923524665,
 1.2830717746819302,
 2.0176604899459636,
 0.544941935097445,
 3.924457603377964,
 4.620930669532201,
 2.5305097147387086,
 1.8009005527899715,
 6.375388720117275,
 8.306725774193088]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a `.map()` method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [20]:
discounted = revenue_minus_tax.map(lambda x: x*0.90)

In [21]:
discounted.take(10)

[0.284370065311722,
 1.1547645972137373,
 1.8158944409513673,
 0.49044774158770055,
 3.5320118430401677,
 4.158837602578981,
 2.277458743264838,
 1.6208104975109745,
 5.737849848105548,
 7.47605319677378]

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff and use a collection method to see the 15 costliest items.

In [22]:
price_items.map(sales_tax).map(lambda x: x*0.90).top(15)

[770.4439037347075,
 760.4982113609099,
 744.6179532361793,
 742.6603850082206,
 739.9402843224501,
 727.5259831595159,
 726.2840736505747,
 725.5657642288173,
 723.8306297128767,
 704.8395060651628,
 701.369231711703,
 694.5815979439706,
 693.8125899223018,
 693.2440317100086,
 688.4596218397954]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [23]:
discounted.toDebugString()

b'(10) PythonRDD[13] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[7] at parallelize at PythonRDD.scala:195 []'

### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use `.flatMap()` rather than a simple `.map()`. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will return a tuple with (original price, post-discount price).

In [24]:
mapped = price_items.map(lambda x: (x, x*0.92*0.90))
print(mapped.count())
print(mapped.take(10))

1000
[(0.34344210786439855, 0.284370065311722), (1.3946432333499241, 1.1547645972137373), (2.1931092282021343, 1.8158944409513673), (0.5923281903233097, 0.49044774158770055), (4.2657147862803955, 3.5320118430401677), (5.022750727752392, 4.158837602578981), (2.750554037759466, 2.277458743264838), (1.9575006008586646, 1.6208104975109745), (6.92977034795356, 5.737849848105548), (9.029049754557704, 7.47605319677378)]


Note that we have 1000 tuples created to our specification. Let's take a look at how `.flatMap()` differs in its implementation. Use the `.flatMap()` method with the same function you created above.

In [25]:
flat_mapped = price_items.flatMap(lambda x: (x, x*0.92*0.90))
print(flat_mapped.count())
print(flat_mapped.take(10))

2000
[0.34344210786439855, 0.284370065311722, 1.3946432333499241, 1.1547645972137373, 2.1931092282021343, 1.8158944409513673, 0.5923281903233097, 0.49044774158770055, 4.2657147862803955, 3.5320118430401677]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use `.flatMap()` rather than `.map()` in order to properly reduce to our specifications. This is not one of those instances, but in the upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the `.filter()` method to select items that bring in more than $300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criterion. In the cell below:
* use a lambda function within a `.filter()` method to meet the consultant's suggestion's specifications. set `RDD = selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [26]:
# use the filter function
selected_items = discounted.filter(lambda x: x>300)

# calculate total remaining in inventory 
selected_items.count()

269

## Reduce

Reduce functions are where you are in some way combing all of the variables that you have mapped out. Here is an example of how a reduce function works when the task is to sum all values:

<img src = "./images/reduce_function.png" width = "600">  


As you can see, the operation is performed within each partition first, after which, the results of the computations in each partition are combined to come up with one final answer.  

Now it's time to figure out how much money BuyStuff would make from selling one of all of its items after they've reduced their inventory. Use the `.reduce()` method with a lambda function to add up all of the values in the RDD. Your lambda function should have two variables. 

In [27]:
selected_items.reduce(lambda x, y: x+y)

124150.35931104477

The time has come for BuyStuff to open up shop and start selling its goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [28]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1, 50), x))

sales_data.take(7)

[(20, 308.2508484010061),
 (20, 303.6355686358445),
 (45, 307.30644059679),
 (44, 312.37654799202227),
 (29, 319.33204199767124),
 (50, 300.8240706871311),
 (27, 341.36173308484655)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called `.reduceByKey()` to perform reducing operations while grouping by keys. After you have calculated the total, use the `.sortBy()` method on the RDD to rank the users from the highest spending to the least spending. 

In [29]:
# calculate how much each user spent
total_spent = sales_data.reduceByKey(lambda x, y: x + y)
total_spent.take(10)

[(20, 6457.506437548453),
 (50, 2627.603818791067),
 (40, 4515.184508021432),
 (10, 1766.947467881192),
 (1, 3322.093023292661),
 (31, 2870.690217646441),
 (21, 1311.5083792148282),
 (11, 3189.4805303825424),
 (41, 742.6603850082206),
 (2, 1648.0928226559863)]

In [30]:
# sort the users from highest to lowest spenders
total_spent.sortBy(lambda x: x[1],ascending = False).collect()

[(20, 6457.506437548453),
 (27, 5054.244363817098),
 (29, 4994.112440772038),
 (44, 4962.128784665479),
 (12, 4631.325650235638),
 (40, 4515.184508021432),
 (4, 4472.81088661592),
 (23, 4133.525794308985),
 (6, 3975.9526688263),
 (36, 3694.556479216165),
 (24, 3550.570492151493),
 (19, 3534.037362601346),
 (1, 3322.093023292661),
 (28, 3242.3544198741383),
 (11, 3189.4805303825424),
 (22, 2944.8226839167987),
 (31, 2870.690217646441),
 (39, 2794.632691357184),
 (3, 2747.3346281079375),
 (17, 2724.155603806672),
 (35, 2679.3306931992997),
 (9, 2668.545017095037),
 (50, 2627.603818791067),
 (49, 2395.9708468795097),
 (5, 2287.059788402787),
 (8, 2162.0477852027266),
 (18, 2091.9931726805153),
 (15, 2081.514184740955),
 (7, 2075.040191951624),
 (43, 2035.852854427705),
 (48, 1953.6431010871033),
 (16, 1907.3494826422652),
 (10, 1766.947467881192),
 (47, 1720.0826704773413),
 (2, 1648.0928226559863),
 (34, 1605.61711988957),
 (14, 1366.7189824727247),
 (21, 1311.5083792148282),
 (45, 1239.

Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [31]:
total_items = sales_data.countByKey()
sorted(total_items.items(),key=lambda kv:kv[1],reverse=True)

[(28, 13),
 (24, 12),
 (27, 12),
 (23, 11),
 (40, 11),
 (42, 10),
 (17, 10),
 (14, 9),
 (2, 8),
 (20, 8),
 (15, 8),
 (22, 7),
 (50, 7),
 (5, 7),
 (44, 7),
 (31, 6),
 (30, 6),
 (38, 6),
 (9, 6),
 (3, 6),
 (26, 6),
 (16, 6),
 (36, 5),
 (12, 5),
 (21, 5),
 (8, 5),
 (29, 5),
 (41, 5),
 (43, 5),
 (48, 4),
 (35, 4),
 (33, 4),
 (1, 4),
 (18, 4),
 (13, 4),
 (39, 4),
 (4, 4),
 (49, 4),
 (34, 3),
 (6, 3),
 (19, 3),
 (25, 2),
 (11, 2),
 (47, 2),
 (45, 1)]

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD and extracting lineage. We also used transformations and actions to perform calculations across RDDs on a distributed setup. In the next lab, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
